In [1]:
# =============================================================================
# FINAL PROJECT PSD - APLIKASI FILTERING AUDIO UNTUK KAGGLE NOTEBOOK
# VERSI 6: PERBAIKAN FINAL UNTUK ERROR 'tuple' object has no attribute 'keys'
# =============================================================================

# --- Impor Library ---
from IPython.display import display, Audio, clear_output
import ipywidgets as widgets
from ipywidgets import HBox, VBox
import numpy as np
import librosa
import librosa.display
from scipy.signal import butter, lfilter
import matplotlib.pyplot as plt
import io

In [2]:
# --- Instalasi Alat Bantu (FFmpeg) ---
print("Mempersiapkan environment...")
!pip install ffmpeg-python -q
clear_output()

# --- Fungsi Inti Pengolahan Sinyal ---
def butter_filter(data, cutoff, fs, btype, order=5):
    nyq = 0.5 * fs
    normal_cutoff = np.array(cutoff) / nyq
    b, a = butter(order, normal_cutoff, btype=btype, analog=False)
    y = lfilter(b, a, data)
    return y

In [3]:
# --- Inisialisasi Widget ---
uploader = widgets.FileUpload(
    accept='.wav, .mp3, .m4a, .ogg, .flac',
    description='1. Pilih Audio',
    button_style='primary',
    multiple=False
)
process_upload_button = widgets.Button(description='2. Proses File', button_style='info', icon='upload')
apply_button = widgets.Button(description='3. Terapkan Filter', button_style='success', icon='cogs')
filter_type_selector = widgets.RadioButtons(options=['Low-pass', 'High-pass', 'Band-pass'], description='Pilih Filter:')
cutoff_input = widgets.Text(value='1000', description='Cutoff (Hz):')
output_area = widgets.Output()

# Global variables
signal = None
sr = None

In [4]:
# --- Fungsi Handler ---
def on_process_upload_clicked(b):
    global signal, sr
    with output_area:
        clear_output(wait=True)
        print("Mencoba memproses file...")
        
        # 'uploader.value' sekarang adalah sebuah TUPLE, bukan dictionary
        uploaded_files_tuple = uploader.value
        
        if not uploaded_files_tuple:
            print("❌ GAGAL: Tidak ada file yang terdeteksi. Pastikan Anda sudah memilih file audio.")
            return
        
        try:
            # === PERBAIKAN UTAMA ADA DI SINI ===
            # Ambil dictionary pertama dari dalam tuple
            file_info = uploaded_files_tuple[0]
            # Ambil nama file dan konten dari dictionary tersebut
            filename = file_info['name']
            file_content = file_info['content']
            # ====================================

            print(f"Membaca file '{filename}'...")
            signal, sr = librosa.load(io.BytesIO(file_content), sr=None)
            print("✅ BERHASIL: File audio berhasil dibaca dan dimuat!")
            
            fig, axs = plt.subplots(2, 1, figsize=(10, 8), tight_layout=True)
            librosa.display.waveshow(signal, sr=sr, ax=axs[0], color='blue')
            axs[0].set_title("Bentuk Gelombang Asli")
            stft = librosa.stft(signal)
            db = librosa.amplitude_to_db(abs(stft), ref=np.max)
            librosa.display.specshow(db, sr=sr, x_axis='time', y_axis='hz', ax=axs[1])
            axs[1].set_title("Spektrogram Asli")
            plt.show()
            
            print("Audio Asli:")
            display(Audio(data=signal, rate=sr))
        except Exception as e:
            print(f"❌ GAGAL saat memproses audio: {e}")

def on_apply_button_clicked(b):
    global signal, sr
    with output_area:
        if signal is None:
            clear_output(wait=True)
            print("❌ PERINGATAN: Anda harus memproses file audio terlebih dahulu! (Klik tombol no. 2)")
            return
        
        try:
            clear_output(wait=True)
            print("Menerapkan filter...")
            btype_map = {'Low-pass': 'low', 'High-pass': 'high', 'Band-pass': 'band'}
            btype = btype_map[filter_type_selector.value]
            cutoff_str = cutoff_input.value
            if btype == 'band':
                cutoffs = [float(f.strip()) for f in cutoff_str.split(',')]
            else: cutoffs = float(cutoff_str)
                
            filtered_signal = butter_filter(signal, cutoffs, sr, btype)
            print("✅ BERHASIL: Filter diterapkan. Menampilkan perbandingan...")
            
            fig, axs = plt.subplots(2, 2, figsize=(12, 8), tight_layout=True)
            librosa.display.waveshow(signal, sr=sr, ax=axs[0, 0], color='blue')
            axs[0, 0].set_title("Asli")
            stft_orig = librosa.amplitude_to_db(abs(librosa.stft(signal)), ref=np.max)
            librosa.display.specshow(stft_orig, sr=sr, x_axis='time', y_axis='hz', ax=axs[1, 0])
            librosa.display.waveshow(filtered_signal, sr=sr, ax=axs[0, 1], color='red')
            axs[0, 1].set_title("Hasil Filter")
            stft_filt = librosa.amplitude_to_db(abs(librosa.stft(filtered_signal)), ref=np.max)
            librosa.display.specshow(stft_filt, sr=sr, x_axis='time', y_axis='hz', ax=axs[1, 1])
            plt.show()
            
            print("Audio Asli:")
            display(Audio(data=signal, rate=sr))
            print("\nAudio Hasil Filter:")
            display(Audio(data=filtered_signal, rate=sr))
        except Exception as e:
            print(f"❌ GAGAL saat menerapkan filter: {e}")


In [5]:
# --- Hubungkan Fungsi ke Tombol ---
process_upload_button.on_click(on_process_upload_clicked)
apply_button.on_click(on_apply_button_clicked)

# --- Atur Tata Letak ---
filter_controls = VBox([widgets.HTML("<h4>Pengaturan Filter:</h4>"), filter_type_selector, cutoff_input])
ui = VBox([
    widgets.HTML("<h3>Aplikasi Interaktif Filtering Audio 🎧</h3>"),
    HBox([uploader, process_upload_button]),
    widgets.HTML("<hr>"),
    HBox([filter_controls, apply_button]),
    widgets.HTML("<hr>"),
    output_area
])

display(ui)